## SED fitting using a functional form SFH;

This notebook will demosntrate how we get SED results with broadband data points using a function form for SFHs.
This example uses the publicly avaialble 3DHST catalog;
https://archive.stsci.edu/prepds/3d-hst/

In [ ]:
%matplotlib inline
%load_ext autoreload

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import ascii,fits
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.table import QTable
import astropy.units as u

from astropy import __version__ as asver
asver

In [ ]:
# https://github.com/mtakahiro/gsf/tree/version1.4
import gsf
print(gsf.__version__)

from gsf.function import get_input
from gsf.gsf import run_gsf_template
from gsf.plot_sed import plot_sed_tau, plot_corner_physparam_frame, plot_corner_physparam_summary
from gsf.plot_sfh import plot_sfh_tau

In [ ]:
## Get photometric catalog;
# 3DHST catalog available at:
# https://archive.stsci.edu/prepds/3d-hst/
field = 'goodss'

# @@ Change this @@
# /Volumes/HD-LXU3/udfcen3/data/3DHST
DIR_3DHST = './CATALOG/%s_3dhst.v4.1.cats/'%(field)
file_phot = DIR_3DHST + 'Catalog/%s_3dhst.v4.1.cat.FITS'%field
fd_phot = fits.open(file_phot)[1].data

file_tran = DIR_3DHST + 'Eazy/%s_3dhst.v4.1.translate'%field
fd_tran = ascii.read(file_tran, format='no_header')


file_pz = DIR_3DHST + 'Eazy/%s_3dhst.v4.1.zout.FITS'%field
fd_pz = fits.open(file_pz)[1].data


In [ ]:
from astropy.table import QTable

file_out = './%s_phot.cat'%field

names = []
for key in fd_tran['col2']:
    names.append('%s'%key)


In [ ]:
tab = {}
tab['id'] = fd_phot['id']
tab['z'] = fd_pz['z_a']

for kk,key in enumerate(fd_tran['col2']):
    try:
        tab[key] = fd_phot[fd_tran['col1'][kk]]
    except:
        print(fd_tran['col1'][kk])
        print('failed.')

t = QTable(tab)
t.write(file_out, overwrite=True, format='ascii')


### Setup gsf

In [ ]:
# Initial setup for gsf.

# Data directory;
DIR_DATA = './'


# Auto load input dictionary;
inputs = get_input()

# change Z;
# Flag;
fplt = 0
inputs['DIR_TEMP'] = './templates/'

# Output directory;
inputs['DIR_OUT'] = './output/'

# If templates exit already, then let's save time.
# (But if you changed metallicity range or age pixels, fplt needs to be 0.)
if os.path.exists('%s/spec_all.asdf'%inputs['DIR_TEMP']):
    fplt = 1

inputs['ID'] = '43114'

# Redshift will be obtained from BB catalog;
inputs['ZGAL'] = 1.90

# Redshift as a free parameter?
inputs['ZMC'] = 0

# Metallicity range, in logZsun;
inputs['ZMIN'] = -1.6
inputs['ZMAX'] = 0.4
inputs['DELZ'] = 0.1
# You can fix metallicity;
#inputs['ZFIX'] = 0.0

# Tau model specification;
inputs['SFH_FORM'] = 4 # delayed tau. 1 for exponential declining model. 
inputs['NPEAK'] = 1 # Number of SFH components.
inputs['AGEMIN'] = -2
inputs['AGEMAX'] = 1.
inputs['DELAGE'] = 0.1
inputs['TAUMIN'] = -1.8
inputs['TAUMAX'] = 1.0
inputs['DELTAU'] = 0.1
inputs['NIMF'] = 1 # Choice for Initial Mass Function. 0=Salpeter, 1=Chabrier, 2=Kroupa, 3=van Dokkum, 4=Dave, 5=tabulated, specified in imf.dat file located in the data directory.
inputs['ADD_NEBULAE'] = 1 # This cannot be done when BPASS is selected.

# Templates;
# Currently in prep.
inputs['BPASS'] = 0

# @@ Change this @@
# Directory path to eazy filter transmission curve files.
inputs['DIR_FILT'] = './GitHub/gsf/example/filter/'

# photometric catalog;
inputs['CAT_BB'] = DIR_DATA + 'goodss_phot.cat'

# Filters;
#inputs['FILTER'] = '205,88,122,79,236,123,124,239,125,203,263,204,264,265,18,19,20,21'
# If any filters need to be skipped.
inputs['SKIPFILT'] = None

# MCMC part;
inputs['NMC'] = 1000 # NMC for the main SED fit
inputs['NMCZ'] = 30 # NMC for the redshift fit
inputs['MC_SAMP'] = 'EMCEE' #ZEUS

# Visual inspection;
# Set to 0 (False), as Notebook cannot show actively iterating plot;
inputs['ZVIS'] = 0

# Initial fit:
inputs['FNELD'] = 'differential_evolution' # nelder, powell


In [ ]:
print(fplt, inputs)


In [ ]:
# Then, run template generate function;
mb = run_gsf_template(inputs, fplt=fplt)
fplt = 1


In [ ]:
# You can write down the input file in an ascii file.
# so you can run the code by e.g.;
# python run_gsf.py file_out 1 
from gsf.function import write_input
write_input(inputs, file_out='gsf_%s_del.input'%inputs['ID'])


### Now, run the whole SED fitting;

In [ ]:
# Import mainbody;
mb = run_gsf_template(inputs, fplt=2)

# If updated any parameter in inputs, run this; 
# For example, if you change mind and want to turn off nebular line;
if True:
    inputs['ADD_NEBULAE'] = 0
    mb.update_input(inputs)

# Already z-fit done, so we can skip z-fit;
skip_fitz = True

# Main fitting routine;
flag_suc = mb.main(cornerplot=True, specplot=1, sigz=1.0, ezmin=0.01, ferr=0, 
                   f_move=False, skip_fitz=skip_fitz, f_shuffle=True)



In [ ]:
# Plot SFH;

# Plot Starforming Main Sequence from Speagle+14?
f_SFMS = True

# Other configuration;
f_silence = False
f_symbol = True
skip_zhist = True
tau_lim = 0.01
tset_SFR_SED = 10
mmax = 300

plot_sfh_tau(mb, f_comp=mb.ftaucomp, fil_path=mb.DIR_FILT, mmax=mmax,
            inputs=mb.inputs, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_silence=f_silence, 
            f_SFMS=f_SFMS, f_symbol=f_symbol, skip_zhist=skip_zhist, tau_lim=tau_lim, tset_SFR_SED=tset_SFR_SED)


In [ ]:
# Plot SED;
plot_sed_tau(mb, fil_path=mb.DIR_FILT,
            figpdf=False, save_sed=True, inputs=mb.inputs, mmax=300,
            f_fill=True, dust_model=mb.dust_model, DIR_TMP=mb.DIR_TMP, f_label=True, f_plot_filter=True)
